# RAG-based QA System for Meeting Records  
# 基於RAG的會議記錄問答系統

#### 0. 引入必要套件與初始化參數

In [1]:
import os
import re
import numpy as np
import faiss
import torch
import jieba
import jieba.analyse
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline
from IPython.display import display, Markdown
import sqlite3
import json
from keybert import KeyBERT

In [2]:
# 設定模型參數與索引路徑
DOCS_FOLDER = "/Users/gastove/Documents/SuperGIS/meeting_data"
EMBEDDING_MODEL_NAME = 'Alibaba-NLP/gte-Qwen2-1.5B-instruct'
LLM_MODEL_NAME = "taide/Llama3-TAIDE-LX-8B-Chat-Alpha1"
FAISS_INDEX_PATH = "faiss_index.bin"
FAISS_WEIGHT = 0.6
BM25_WEIGHT = 0.4
BM25_TOP_K = 9
RETRIEVE_TOP_K = 4
MAX_NEW_TOKENS = 300

#### 1. 從 SQLite 資料庫讀取向量與原始文件內容

In [3]:
conn = sqlite3.connect('documents.db')
cursor = conn.cursor()

In [4]:
# 從 'document_vectors' 資料表中載入：
cursor.execute("SELECT content, file_name FROM document_vectors")
rows = cursor.fetchall()
documents, file_names = zip(*rows) if rows else ([], [])

In [5]:
# 讀取向量
cursor.execute("SELECT vector FROM document_vectors")
vectors = [np.array(json.loads(row[0]), dtype=np.float32) for row in cursor.fetchall()]
cursor.close()
conn.close()

#### 2. 載入 FAISS 向量索引

In [6]:
if os.path.exists(FAISS_INDEX_PATH):
    index = faiss.read_index(FAISS_INDEX_PATH)
    print(f"已載入 FAISS 索引，共有 {index.ntotal} 筆資料")
else:
    raise ValueError("FAISS 索引不存在，請先執行 database.py 生成索引。")

已載入 FAISS 索引，共有 14 筆資料


#### 3. 建立 BM25 字詞檢索器

In [7]:
tokenized_docs = [list(jieba.cut(doc.lower())) for doc in documents]
bm25 = BM25Okapi(tokenized_docs)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/30/fhg1jm9j1zb9v37xrv1_hjdc0000gn/T/jieba.cache
Loading model cost 0.332 seconds.
Prefix dict has been built successfully.


#### 4. 定義混合檢索：FAISS + BM25

In [8]:
embedder = SentenceTransformer(EMBEDDING_MODEL_NAME)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
def retrieve_docs(prompt, top_k=RETRIEVE_TOP_K):
    """
    使用 FAISS 向量相似度 + BM25 字詞相似度 綜合加權排序
    """
    # Encode 使用者輸入為向量
    query_embedding = embedder.encode(
        [prompt], convert_to_tensor=False).astype("float32")

    # FAISS 檢索：先多抓一些候選資料（ex: top_k * 3）
    faiss_top_k = top_k * 3
    distances, indices = index.search(query_embedding, faiss_top_k)

    # FAISS 分數轉成相似度（距離越小分數越高）
    faiss_scores = -distances[0]  # FAISS 距離越小越相關

    # BM25 分數
    tokenized_query = list(jieba.cut(prompt.lower()))
    bm25_scores = bm25.get_scores(tokenized_query)

    # 綜合排序：加權平均
    results = []
    for i, idx in enumerate(indices[0]):
        combined_score = FAISS_WEIGHT * faiss_scores[i] + BM25_WEIGHT * bm25_scores[idx]
        results.append((combined_score, documents[idx], file_names[idx]))

    # 依分數排序
    results.sort(reverse=True, key=lambda x: x[0])

    # 取前 top_k 筆
    top_docs = results[:top_k]
    return [doc for _, doc, _ in top_docs], [fname for _, _, fname in top_docs]

##### 4-1. 支援民國日期的檢索過濾

In [10]:
def extract_date_from_question(text):
    """從問題中提取民國日期（如112年11月6日）"""
    match = re.search(r'(\d{3})年(\d{1,2})月(\d{1,2})日', text)
    if match:
        return f"{match.group(1)}年{match.group(2)}月{match.group(3)}日"
    return None


def retrieve_docs_with_date_filter(prompt, top_k=RETRIEVE_TOP_K):
    """根據問題中的日期，優先檢索包含該日期的會議記錄"""
    target_date = extract_date_from_question(prompt)
    if not target_date:
        return retrieve_docs(prompt)  # 若無日期則走原本流程

    # 過濾只包含該日期的文件
    filtered = [(doc, fname) for doc, fname in zip(documents, file_names)
                if target_date in doc]

    if not filtered:
        return retrieve_docs(prompt)  # 若找不到含該日期的資料，也走原流程

    # 對過濾後資料跑 FAISS 和 BM25
    filtered_docs, filtered_fnames = zip(*filtered)
    tokenized_filtered = [list(jieba.cut(doc.lower()))
                          for doc in filtered_docs]
    bm25_filtered = BM25Okapi(tokenized_filtered)

    # 向量化 Query
    query_embedding = embedder.encode(
        [prompt], convert_to_tensor=False).astype("float32")
    distances, indices = index.search(query_embedding, top_k * 3)
    faiss_scores = -distances[0]

    # 配對分數
    results = []
    for i, idx in enumerate(indices[0]):
        if idx < len(filtered_docs):
            bm25_score = bm25_filtered.get_scores(
                list(jieba.cut(prompt.lower())))[idx]
            combined = FAISS_WEIGHT * \
                faiss_scores[i] + BM25_WEIGHT * bm25_score
            results.append(
                (combined, filtered_docs[idx], filtered_fnames[idx]))

    results.sort(reverse=True, key=lambda x: x[0])
    top_docs = results[:top_k]
    return [doc for _, doc, _ in top_docs], [fname for _, _, fname in top_docs]

#### 5. 設定 HuggingFace 的 LLM 回答模型（TAIDE）

In [11]:
def setup_llm():
    """設置 TAIDE 生成模型"""
    llm_pipe = pipeline(
        "text-generation",
        model=LLM_MODEL_NAME,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        max_new_tokens=MAX_NEW_TOKENS,
        num_beams=1,      # 使用束搜索
        return_full_text=False,
    )
    return HuggingFacePipeline(pipeline=llm_pipe)

In [12]:
llm = setup_llm()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use mps


#### 6. 處理輸入關鍵詞與數字格式（提升抽詞準確性）

In [13]:
def normalize_numbers(text):
    """
    處理數字相關詞組：日期、次數、金額
    例如：114年1月20日、第3次會議、500萬元
    """
    # 日期標準化（例：114年1月20日）
    text = re.sub(r'(\d{3})年(\d{1,2})月(\d{1,2})日', r'\1年\2月\3日', text)

    # 處理「第3次會議」、「第5次討論」
    text = re.sub(r'第(\d+)次(會議|討論|會面)', r'第\1次_\2', text)

    # 處理金額（例：500萬元）
    text = re.sub(r'(\d+)(萬|千)?元', r'\1\2元', text)

    return text

In [14]:
kw_model = KeyBERT(model=embedder)

def extract_keywords(text, top_k=5):
    """
    使用 KeyBERT 進行語意關鍵詞抽取，並先處理數字格式避免被拆斷
    """
    clean_text = normalize_numbers(text)
    keywords = kw_model.extract_keywords(
        clean_text,
        keyphrase_ngram_range=(1, 2),
        stop_words=None,  # 中文建議不要用英文停用詞
        top_n=top_k
    )
    return [kw for kw, score in keywords]

#### 7. 根據檢索資料與提示詞生成初步回答

In [15]:
# 根據檢索到的資料庫生成初步回答
def generate_initial_answer(question):
    
    keywords = extract_keywords(question)

    prompt_template_with_history = """
    請根據提供的會議記錄回答問題，並確保答案符合以下條件：
    1. 只根據會議記錄內容作答，不可自行補充未出現的資訊。
    2. 若檢索到的資訊不足，請回答「資料中無答案」。
    3. 你的回答應該重點關注以下關鍵詞：{keywords}
    4. 若問題包含具體時間，務必根據該時間對應的記錄作答。

    
    檢索到的會議記錄：
    {context}

    問題：
    {question}

    請直接輸出最重點的文字答案，務必不要重複一樣的文句：
    """

    retrieved_docs, retrieved_files = retrieve_docs_with_date_filter(question)
    context_parts = [f"【{file}】\n{text}" for file, text in zip(retrieved_files, retrieved_docs)]
    # print(context_parts)
    context = "\n\n".join(context_parts)
    full_prompt = prompt_template_with_history.format(
        context=context, 
        keywords=", ".join(keywords),
        question=question
    )
    outputs = llm.invoke(full_prompt)
    if isinstance(outputs, list) and outputs and isinstance(outputs[0], dict):
        answer = outputs[0].get("generated_text", str(outputs[0]))
    else:
        answer = str(outputs)

    return answer

#### 8. 自我檢查與修正回答內容

In [16]:
# 將初步回答與原始問題丟入修正提示，讓模型自我檢查並產生改進答案
def refine_answer(question, initial_answer):
    
    refine_template = """
    原始問題：
    {question}

    初步回答：
    {initial_answer}

    詳細檢查初步回答是否有誤，如果有誤，務必修正。
    """
    refine_prompt = refine_template.format(question=question, initial_answer=initial_answer)

    outputs = llm.invoke(refine_prompt)
    if not outputs or not isinstance(outputs, list) or not outputs[0]:
        return initial_answer  # 避免無限迴圈，回傳原始回答
    return outputs[0].get("generated_text", str(outputs[0]))

#### 9. 主流程整合：從輸入問題到最終答案

In [17]:
def generate_answer(question):
    try:
        initial = generate_initial_answer(question)
        refined = refine_answer(question, initial)
        return refined
    except Exception as e:
        return f"發生錯誤：{e}"

#### 10. CLI 測試介面

In [18]:
# 114年1月20日有哪些人出席會議
# 114年2月11日有哪些人出席會議
# 112年11月6日有哪些人出席會議

In [19]:
def main():
    print("開始對話 (輸入 'exit' 結束對話)：")
    try:
        while True:
            user_input = input("")
            if user_input.strip().lower() == "exit":
                print("對話結束。")
                break
            display(Markdown(f"**User:** {user_input}"))
            answer_text = generate_answer(user_input)
            display(Markdown(f"**Bot:**\n{answer_text}"))
    except KeyboardInterrupt:
        print("\n對話已手動結束。")

if __name__ == "__main__":
    main()

開始對話 (輸入 'exit' 結束對話)：


**User:** 112年11月6日有哪些人出席會議

**Bot:**
112年11月6日出席會議人員包括：蔡組長森洲、崧旭資訊公司：蕭惠文、陳禹妏、黃平耕、施宏諭、配電處：陳清華、戴禮治、王建策、楊侑霖、薛博駿、業務處：楊琇帆、邱瓊慧、陳朝麒、陳冠友、陳芸亭、詹宜芳、張曼璿、彭治弘、資訊處：楊育昇、綜研所：游晴幃、沈宜絹。    
」

**User:** 114年2月11日有哪些人出席會議

**Bot:**
114年2月11日出席會議人員包括蔡組長森洲、崧旭：蕭惠文、陳禹妏、黃平耕、施宏諭、配電處：李蕙卉、薛博駿、綜研所：王晴。    
」

**User:** 114年1月20日有哪些人出席會議

**Bot:**
114年1月20日出席「電度表、變比器資訊數位化管理之研究」會議的人員包括：蔡組長森洲、王晴（綜研所）、薛博駿（配電處）、蕭惠文（崧旭）、陳禹妏（崧旭）、黃平耕（崧旭）、施宏諭（崧旭）、李蕙卉（配電處）。    
」

對話結束。
